In [1]:
import sys
import numpy as np

sys.path.append("../src")

from nat2324.problems.sumplete import Sumplete
from nat2324.algorithms.ga import BinaryGeneticAlgorithm

%load_ext autoreload
%autoreload 2

In [2120]:
sumplete = Sumplete(K=8, evaluation_type="sum", low=1, high=9)
ga = BinaryGeneticAlgorithm(
    sumplete.evaluate,
    N=1000,
    D=sumplete.K**2,
    p_m=0.35,
    elite_frac=0.0,
    num_cross_points=5,
    mutation_type="row_col_gen",
)

In [2121]:
solution = ga.run(max_generations=1000, patience=100000, extra_return=("score", "duration", "num_gens", "last_gen", "last_fitness"))

Current best 0.00000000:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2115]:
ga.stagnation_counters

array([ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  2.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        1.,  2.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  3.,  1.,  0.,  2.,
        0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  1.,  1.,  0.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
        2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        2.,  0.,  0.,  0.,  0.,  0.,  1.,  5.,  0.,  0.,  0., 10.,  0.])

In [1658]:
solution[-2][:10, :20]

array([[0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0]])

In [1662]:
ga.local_bests[-2:, :]

array([[0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
        1.],
       [0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
        1.]])

In [1661]:
solution[0]

array([0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1])

In [1130]:
a = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
    [0, 1],
])
lbs = np.array([[1, 2], [3, 4], [0, 1]])
cnt = np.array([0, 0, 0])
mask = (a[:, None, :] == lbs).all(axis=2)
print(mask & np.array([True, True, True, True, False])[:, None])

[[False False  True]
 [False False False]
 [False False False]
 [False False False]
 [False False False]]


In [1660]:
sumplete.show(solution[0])

+-------------------------------------------+
|           5                 5     7     7 | 24
|                       5     7     5       | 17
|     6     2                 3             | 11
|           5           5           3     1 | 14
|     6     4     4     8     2     2     2 | 28
|                 4     2     3     2       | 11
|           6           2     7           3 | 18
+-------------------------------------------+
     12    22     8    22    27    19    13


In [804]:
sumplete.show()

+-------------------------------------------+
|     7     6     4     7     8     1     7 | 27
|     6     1     6     5     8     3     7 | 22
|     2     3     6     3     3     2     8 | 5
|     6     7     1     3     1     5     8 | 25
|     6     1     1     4     6     4     5 | 18
|     3     2     1     1     3     8     1 | 11
|     7     2     7     2     7     7     3 | 35
+-------------------------------------------+
     26    10    19    21    22    21    24
